In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

/home/cent/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cent/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cent/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cent/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [38]:

pickle_file = 'notMNIST.pickle'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)
                        
                         

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [39]:
image_size = 28
num_labels = 10
num_channels = 1
import numpy as np 

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)
                                              

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [35]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

In [47]:
batch_size = 16
patch_size = 5 
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
                                
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
                                
    def model(data):
        #conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        #hidden = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(data, 2, 2, padding='SAME')
        hidden = tf.nn.relu(pool + layer1_biases)
        #conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        #hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(hidden, 2, 2, padding='SAME')
        hidden = tf.nn.relu(pool + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
                                
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [48]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size),:]
        feed_dict = {tf_train_dataset :batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 5 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.610463
Minibatch accuracy: 12.5%
Validation accuracy: 10.4%
Minibatch loss at step 5: 2.156919
Minibatch accuracy: 12.5%
Validation accuracy: 15.2%
Minibatch loss at step 10: 2.292690
Minibatch accuracy: 12.5%
Validation accuracy: 12.9%
Minibatch loss at step 15: 2.304640
Minibatch accuracy: 6.2%
Validation accuracy: 10.4%
Minibatch loss at step 20: 2.304074
Minibatch accuracy: 6.2%
Validation accuracy: 10.8%
Minibatch loss at step 25: 2.265502
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 30: 2.264918
Minibatch accuracy: 18.8%
Validation accuracy: 11.3%
Minibatch loss at step 35: 2.309261
Minibatch accuracy: 6.2%
Validation accuracy: 17.0%
Minibatch loss at step 40: 2.269516
Minibatch accuracy: 6.2%
Validation accuracy: 11.9%
Minibatch loss at step 45: 2.280427
Minibatch accuracy: 18.8%
Validation accuracy: 12.7%
Minibatch loss at step 50: 2.245216
Minibatch accuracy: 25.0%
Validation accuracy: 10.0%
Minibatch loss a

Validation accuracy: 35.6%
Minibatch loss at step 465: 1.447958
Minibatch accuracy: 37.5%
Validation accuracy: 49.2%
Minibatch loss at step 470: 1.385489
Minibatch accuracy: 56.2%
Validation accuracy: 46.6%
Minibatch loss at step 475: 1.804746
Minibatch accuracy: 37.5%
Validation accuracy: 58.3%
Minibatch loss at step 480: 0.995762
Minibatch accuracy: 62.5%
Validation accuracy: 56.5%
Minibatch loss at step 485: 1.437963
Minibatch accuracy: 50.0%
Validation accuracy: 52.1%
Minibatch loss at step 490: 1.738839
Minibatch accuracy: 43.8%
Validation accuracy: 53.1%
Minibatch loss at step 495: 1.757032
Minibatch accuracy: 62.5%
Validation accuracy: 63.2%
Minibatch loss at step 500: 0.799494
Minibatch accuracy: 68.8%
Validation accuracy: 63.1%
Minibatch loss at step 505: 1.966976
Minibatch accuracy: 50.0%
Validation accuracy: 45.2%
Minibatch loss at step 510: 1.385166
Minibatch accuracy: 50.0%
Validation accuracy: 38.3%
Minibatch loss at step 515: 0.940366
Minibatch accuracy: 68.8%
Validation

Validation accuracy: 70.7%
Minibatch loss at step 925: 0.952022
Minibatch accuracy: 68.8%
Validation accuracy: 71.0%
Minibatch loss at step 930: 1.450204
Minibatch accuracy: 50.0%
Validation accuracy: 71.8%
Minibatch loss at step 935: 0.928469
Minibatch accuracy: 62.5%
Validation accuracy: 70.7%
Minibatch loss at step 940: 1.105459
Minibatch accuracy: 87.5%
Validation accuracy: 75.7%
Minibatch loss at step 945: 0.643921
Minibatch accuracy: 87.5%
Validation accuracy: 68.3%
Minibatch loss at step 950: 1.241291
Minibatch accuracy: 43.8%
Validation accuracy: 60.9%
Minibatch loss at step 955: 0.753239
Minibatch accuracy: 75.0%
Validation accuracy: 72.6%
Minibatch loss at step 960: 1.049808
Minibatch accuracy: 68.8%
Validation accuracy: 71.5%
Minibatch loss at step 965: 1.500360
Minibatch accuracy: 62.5%
Validation accuracy: 70.3%
Minibatch loss at step 970: 0.924379
Minibatch accuracy: 75.0%
Validation accuracy: 71.5%
Minibatch loss at step 975: 1.294251
Minibatch accuracy: 50.0%
Validation

In [41]:
?tf.nn.max_pool

W0923 20:09:03.223753 140401769748288 deprecation_wrapper.py:119] From /home/cent/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:1556: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

